# Notes for next work

In [1]:
##NOTES

## This all runs on the assumptions that the "closest tiebreaker" is sound for hitting a minimal cover.
## That could be the math portion of the capstone, talk to Dr. B
## Inherently, the choose next vertex has a secondary tiebreaker of smallest label instead of randomly picking from
##     minimized distance. Probaly should change.


## Ask Dr B for smaller solved 4d solids


## For 600 - cell
## 

# Theory and Thought process

In [2]:
##Idea that faces touched is = edges vertex is on, seems to hold for all 3d platonic solids
    ## How to find every vertex that shares an edge, used here and in distance calculation
        ## Shares an edge if they share 2 or more faces
        ## How to find shared faces
## Just brute forcing through the faces listings in the inputMatrix works, but can be super slow
    ##For 600-face, around 200,000 actions, 2.5 to find certain vertex in face, scan 600 faces for 
    ##    first vertex, then that subset for second vertex (20), then 120 vertices
## Binary matrix probably easier, less searching
    ## Would just find each face that has the first vertex, then find each that has 2nd in that subset
        ## 600, then 20, by 120? Down below 100,000 actions
    ## Around 2 or 3 times faster
    
## Calculate distance of vertices to faces
    ## Find distance from a certain vertex to all vertices on a face, take minimum
    ## Needs to be upfront, not on call
    ## 1 unit away = shares an edge with a vertex on the face
        ## Share an edge if they share more than 1 face (ASK DR B IF THIS IS TRUE)**********************************
    ## 2 units away = shares an edge with a vertex that is 1 unit away from the face
    ## Need to use a path finding algorithm to find minimal distance from each vertex to each other vertex
        ## Ew, thats a ton of computation
    ## Store how?
        ## Could be a 2d array with an int value for number of edges away
            #rows, cols = (maxVert, maxVert)
            #print (rows, cols)
            #vertexListOfTouchingFaces = [[0 for i in range(cols)] for j in range(rows)]
            #print (vertexListOfTouchingFaces)
        ## Could be a 2d array where the cells are lists of vertices that are a certain distance away
        ## If just using vertex to vertex and doing multiple search per face instead of computations, should
        ##   probably use the first one
        ## Ex: d4
        ##   1 2 3 4
        ## 1 0 1 1 1
        ## 2 1 0 1 1
        ## 3 1 1 0 1
        ## 4 1 1 1 0
        
        ## Ex: d6
        ##   1 2 3 4 5 6 7 8
        ## 1 0 1 2 1 1 2 3 2
        ## 2 1 0 1 2 2 1 2 3 
        ## 3 2 1 0 1 3 2 1 2
        ## 4 1 2 1 0 2 3 2 1
        ## 5 1 2 3 2 0 1 2 1
        ## 6 2 1 2 3 1 0 1 2
        ## 7 3 2 1 2 2 1 0 1
        ## 8 2 3 2 1 1 2 1 0
        
        ## Ex: d8
        ##   1 2 3 4 5 6
        ## 1 0 1 1 1 1 2
        ## 2 1 0 1 2 1 1
        ## 3 1 1 0 1 2 1
        ## 4 1 2 1 0 1 1
        ## 5 1 1 2 1 0 1
        ## 6 2 1 1 1 1 0
        
        ## These matrices should always be symmetric on diagonal
        
        ## Filling these out would be like a tree, make the identity 0, fill out everything thats 1 away,
        ## Fill out everything 2 away, so on
        ## Finding stuff 2 away, find everything 1 away from vertices already 1 away, excluding what's already listed
        ## ............. 3 ..................... 1 .......................... 2 .....................................
        
        
        ## This means the creation of this matrix won't be row by row completely
        ## Easier to go row by row hitting everything 1 away, then row by row hitting everything 2 away
            ## Do this since for 2 away, need info of whats 1 away from every other vertex
            ## Read from notebook if can't visualize

# Imports

In [3]:
##IMPORTS
import numpy as np
from numpy import *
import copy

# Take in or create input

In [4]:

## FOR THE BIG 600 CELL ONE
import csv

#with open("UseThisOne.csv") as file_name:
    #inputMatrix = np.loadtxt(file_name, dtype = 'int', delimiter=",")

    
## FOR SMALLER TESTING SOLIDS
inputMatrix = array([[1,2,3,10],
               [2,3,4,11],
               [3,4,5,12],
               [4,5,6,13],
               [5,6,7,14],
               [6,7,8,15],
               [1,7,8,16],
               [1,2,8,9],
               [1,2,11,16],
               [2,3,9,12],
               [3,4,10,13],
               [4,5,11,14],
               [5,6,12,15],
               [6,7,13,16],
               [7,8,9,14],
               [1,8,10,15],
               [1,10,13,16],
               [2,9,11,14],
               [3,10,12,15],
               [4,11,13,16],
               [5,9,12,14],
               [6,10,13,15],
               [7,11,14,16],
               [8,9,12,15],
               
              ])

    
print(inputMatrix)


[[ 1  2  3 10]
 [ 2  3  4 11]
 [ 3  4  5 12]
 [ 4  5  6 13]
 [ 5  6  7 14]
 [ 6  7  8 15]
 [ 1  7  8 16]
 [ 1  2  8  9]
 [ 1  2 11 16]
 [ 2  3  9 12]
 [ 3  4 10 13]
 [ 4  5 11 14]
 [ 5  6 12 15]
 [ 6  7 13 16]
 [ 7  8  9 14]
 [ 1  8 10 15]
 [ 1 10 13 16]
 [ 2  9 11 14]
 [ 3 10 12 15]
 [ 4 11 13 16]
 [ 5  9 12 14]
 [ 6 10 13 15]
 [ 7 11 14 16]
 [ 8  9 12 15]]


# Find total number of vertices

In [5]:
## Find number of vertices
inputMatrixWidth= len(inputMatrix[0])
maxVert = 0
for x in inputMatrix:
    if x[inputMatrixWidth-1] > maxVert:
        maxVert = x[inputMatrixWidth-1]

print(maxVert)

16


# Create master array for which faces have and haven't been covered

In [6]:
## Create Master Array of Uncovered Faces
## Will be removed from throughout process
masterUncoveredFaces = copy.deepcopy(inputMatrix)
print(masterUncoveredFaces)
print("")

##Create Master Array for covered faces, starts empty, fills to be inputMatrix

rows, cols = (1, inputMatrixWidth)
masterCoveredFaces = [[-1 for i in range(cols)] for j in range(rows)]
#print(vertexToVertexDistance)


print(masterCoveredFaces)

[[ 1  2  3 10]
 [ 2  3  4 11]
 [ 3  4  5 12]
 [ 4  5  6 13]
 [ 5  6  7 14]
 [ 6  7  8 15]
 [ 1  7  8 16]
 [ 1  2  8  9]
 [ 1  2 11 16]
 [ 2  3  9 12]
 [ 3  4 10 13]
 [ 4  5 11 14]
 [ 5  6 12 15]
 [ 6  7 13 16]
 [ 7  8  9 14]
 [ 1  8 10 15]
 [ 1 10 13 16]
 [ 2  9 11 14]
 [ 3 10 12 15]
 [ 4 11 13 16]
 [ 5  9 12 14]
 [ 6 10 13 15]
 [ 7 11 14 16]
 [ 8  9 12 15]]

[[-1, -1, -1, -1]]


# Creating the Dictionary of each face that touches each vertex

In [7]:
## Create list of faces touched by each vertex, as well as copy for strictly uncovered

## How to know how many faces each vertex will touch? **************************
    ## For platonic solid, equal for all, so find max faces for one, for all
    ## Just make it a dictionary, probably better

vertexDictOfTouchedFaces = {
    
}

for nextVertex in range(maxVert):
    vertexDictOfTouchedFaces[nextVertex+1] = []
    
## Fill in the dictionary
for currentFace in inputMatrix:
    for currentVertex in currentFace:
            vertexDictOfTouchedFaces[currentVertex].append(currentFace)
            
print(vertexDictOfTouchedFaces)


##Create the dictionary for each uncovered face, will be removed from throughout the process
vertexDictOfTouchingUncoveredFaces = copy.deepcopy(vertexDictOfTouchedFaces)

print(vertexDictOfTouchedFaces[1])
print(len(vertexDictOfTouchedFaces[1]))
    

    

{1: [array([ 1,  2,  3, 10]), array([ 1,  7,  8, 16]), array([1, 2, 8, 9]), array([ 1,  2, 11, 16]), array([ 1,  8, 10, 15]), array([ 1, 10, 13, 16])], 2: [array([ 1,  2,  3, 10]), array([ 2,  3,  4, 11]), array([1, 2, 8, 9]), array([ 1,  2, 11, 16]), array([ 2,  3,  9, 12]), array([ 2,  9, 11, 14])], 3: [array([ 1,  2,  3, 10]), array([ 2,  3,  4, 11]), array([ 3,  4,  5, 12]), array([ 2,  3,  9, 12]), array([ 3,  4, 10, 13]), array([ 3, 10, 12, 15])], 4: [array([ 2,  3,  4, 11]), array([ 3,  4,  5, 12]), array([ 4,  5,  6, 13]), array([ 3,  4, 10, 13]), array([ 4,  5, 11, 14]), array([ 4, 11, 13, 16])], 5: [array([ 3,  4,  5, 12]), array([ 4,  5,  6, 13]), array([ 5,  6,  7, 14]), array([ 4,  5, 11, 14]), array([ 5,  6, 12, 15]), array([ 5,  9, 12, 14])], 6: [array([ 4,  5,  6, 13]), array([ 5,  6,  7, 14]), array([ 6,  7,  8, 15]), array([ 5,  6, 12, 15]), array([ 6,  7, 13, 16]), array([ 6, 10, 13, 15])], 7: [array([ 5,  6,  7, 14]), array([ 6,  7,  8, 15]), array([ 1,  7,  8, 16])

# Create Dictionary Of Which Vertices a Single Vertex shares edges with

In [8]:
##Create something that stores every vertex, and each other vertex it shares an edge with
    ## shares edge if it shares 2 faces
    ##Look in vertexDictOfTouchedFaces, keep count of each time each vertex appears. If > 1 and not the original, edge
##What to store in, dict doesn't seem to be working well, hard with arrays as values
    ## Dict for now
    
vertexDictOfSharedEdges = {}

for nextVertex in range(maxVert):
    vertexDictOfSharedEdges[nextVertex+1] = []

vertexDictOfVerticesOnEdge = {    
}

for nextVertex in range(maxVert):
    vertexDictOfVerticesOnEdge[nextVertex+1] = []

rows, cols = (maxVert, maxVert)
vertexSharedFacesCount = [[0 for i in range(cols)] for j in range(rows)]
#print(len(vertexSharedFacesCount)*len(vertexSharedFacesCount[0]))



for currentVertex in vertexDictOfTouchedFaces:
    
    #print(currentVertex, vertexDictOfTouchedFaces[currentVertex])
    for currentFace in vertexDictOfTouchedFaces[currentVertex]:
        for vertexOnFace in currentFace:
            vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] += 1
            
            ## This if statement is the problem, assumes points only connect on 2 faces
                ## Or maybe not
            
            if vertexSharedFacesCount[currentVertex-1][vertexOnFace-1] > 1 and vertexOnFace not in vertexDictOfSharedEdges[currentVertex]:
                    if vertexOnFace != currentVertex:
                        vertexDictOfSharedEdges[currentVertex].append(vertexOnFace)
    
    ##PUT THIS INSIDE LOOP ABOVE, SO TO ACCESS THE VALUE OF vertexOnFace
    ## Also, use number of faces each vertex is on to eliminate vertices pairing with self
    ##for x in vertexSharedFacesCount[currentVertex-1]:
        ##if x > 1:
            ##print(x, currentVertex)
            


#print(vertexSharedFacesCount)
#print(vertexDictOfSharedEdges)

tempVertexDictOfSharedEdges = copy.deepcopy(vertexDictOfSharedEdges)
#print(tempVertexDictOfSharedEdges)

#Once this removes an element from the dictionary, it doesn't check the next value. use temp dictionary
for currentVertex in vertexDictOfSharedEdges:
    for connectedVertex in vertexDictOfSharedEdges[currentVertex]:
        #print(currentVertex, connectedVertex)
        if connectedVertex == currentVertex:
            tempVertexDictOfSharedEdges[currentVertex].remove(connectedVertex)
            #print("removed", connectedVertex)
            
print(vertexDictOfSharedEdges)

vertexDictOfSharedEdges = copy.deepcopy(tempVertexDictOfSharedEdges)
    
print(vertexDictOfSharedEdges)  



{1: [2, 8, 16, 10], 2: [3, 1, 11, 9], 3: [2, 4, 12, 10], 4: [3, 5, 13, 11], 5: [4, 6, 14, 12], 6: [5, 7, 15, 13], 7: [6, 8, 16, 14], 8: [7, 1, 9, 15], 9: [2, 8, 14, 12], 10: [3, 1, 13, 15], 11: [2, 4, 14, 16], 12: [3, 5, 15, 9], 13: [4, 6, 10, 16], 14: [5, 7, 9, 11], 15: [6, 8, 10, 12], 16: [1, 7, 13, 11]}
{1: [2, 8, 16, 10], 2: [3, 1, 11, 9], 3: [2, 4, 12, 10], 4: [3, 5, 13, 11], 5: [4, 6, 14, 12], 6: [5, 7, 15, 13], 7: [6, 8, 16, 14], 8: [7, 1, 9, 15], 9: [2, 8, 14, 12], 10: [3, 1, 13, 15], 11: [2, 4, 14, 16], 12: [3, 5, 15, 9], 13: [4, 6, 10, 16], 14: [5, 7, 9, 11], 15: [6, 8, 10, 12], 16: [1, 7, 13, 11]}


# Fill out matrix of vertex to vertex distances

In [9]:
rows, cols = (maxVert, maxVert)
vertexToVertexDistance = [[999 for i in range(cols)] for j in range(rows)]
#print(vertexToVertexDistance)

for i in range(0,maxVert):
    vertexToVertexDistance[i][i] = 0

#print( vertexToVertexDistance)

edgesAway = 1

while np.amax(vertexToVertexDistance) == 999:
    for vertex in vertexDictOfSharedEdges:
        for secondVertex in vertexDictOfSharedEdges:
            if vertexToVertexDistance[vertex-1][secondVertex-1] == edgesAway-1:
                #print(secondVertex, " is ", edgesAway, " away")
                for vertXAway in vertexDictOfSharedEdges[secondVertex]:
                    if vertexToVertexDistance[vertex-1][vertXAway-1] == 999:
                        vertexToVertexDistance[vertex-1][vertXAway-1] = edgesAway
    edgesAway += 1
print(vertexToVertexDistance)

print(vertexToVertexDistance)
#print(np.amax(vertexToVertexDistance))

[[0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2, 3, 2, 1], [1, 0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2, 3, 2], [2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2, 3], [3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2], [4, 3, 2, 1, 0, 1, 2, 3, 2, 3, 2, 1, 2, 1, 2, 3], [3, 4, 3, 2, 1, 0, 1, 2, 3, 2, 3, 2, 1, 2, 1, 2], [2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 2, 3, 2, 1, 2, 1], [1, 2, 3, 4, 3, 2, 1, 0, 1, 2, 3, 2, 3, 2, 1, 2], [2, 1, 2, 3, 2, 3, 2, 1, 0, 3, 2, 1, 4, 1, 2, 3], [1, 2, 1, 2, 3, 2, 3, 2, 3, 0, 3, 2, 1, 4, 1, 2], [2, 1, 2, 1, 2, 3, 2, 3, 2, 3, 0, 3, 2, 1, 4, 1], [3, 2, 1, 2, 1, 2, 3, 2, 1, 2, 3, 0, 3, 2, 1, 4], [2, 3, 2, 1, 2, 1, 2, 3, 4, 1, 2, 3, 0, 3, 2, 1], [3, 2, 3, 2, 1, 2, 1, 2, 1, 4, 1, 2, 3, 0, 3, 2], [2, 3, 2, 3, 2, 1, 2, 1, 2, 1, 4, 1, 2, 3, 0, 3], [1, 2, 3, 2, 3, 2, 1, 2, 3, 2, 1, 4, 1, 2, 3, 0]]
[[0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2, 3, 2, 1], [1, 0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2, 3, 2], [2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2, 3], [3, 2, 1, 0, 1, 2, 3, 4, 3, 2, 1, 2, 1, 2, 3, 2]

# Calculate the distance from each maxUntouchedVertex to a covered face

In [10]:
## Create function for running calucations on each vertex in an input list
## Simply returns which of the input vertices is closest

## Premake the matrix of distances

## When called, search through covered faces
## Pull out every vertex in the covered faces sets

## For each maxUntouchedVertex, find distance each vertex in covered faces sets, take min
## Store min as distance from covered faces

## Find maxUntouchedVertex with smallest distance from covered faces

## Return that maxUntouchedVertex

def findNextVertex(listOfMaxedFaceVerts):
    
    ## Fill out all vert touching a covered face
    verticesTouchingCoveredFaces = []
    for coveredFace in masterCoveredFaces:
        for vertex in coveredFace:
            if vertex not in verticesTouchingCoveredFaces:
                #print( vertex, "isn't in verticesTouchingCoveredFaces")
                verticesTouchingCoveredFaces.append(vertex)
                #print("verticesTouchingCoveredFaces is now ", verticesTouchingCoveredFaces)
    dictOfMaxedVertDistanceToFace = {}            
    for vertex in maxUntouchedVerts:
        dictOfMaxedVertDistanceToFace[vertex] = []
        closestDistance = 999999
        for vertOnFace in verticesTouchingCoveredFaces:
            if vertexToVertexDistance[vertex-1][vertOnFace -1] < closestDistance:
                closestDistance = vertexToVertexDistance[vertex-1][vertOnFace -1]
        #print(vertex, closestDistance)
        dictOfMaxedVertDistanceToFace[vertex] = closestDistance
            
        
    #print(dictOfMaxedVertDistanceToFace)

    ## Find maxUntouchedVertex with smallest distance from covered faces
    closestVert = 9999
    closestVertDist = 9999
    
    for vertex in dictOfMaxedVertDistanceToFace:
        if dictOfMaxedVertDistanceToFace[vertex] < closestVertDist:
            closestVert = vertex
            closestVertDist = dictOfMaxedVertDistanceToFace[vertex]
            #print("New closest is ", vertex)
    return(closestVert)

# Randomly select starting vertex

In [11]:
## Select Random Starting vertex
startingVertex = random.randint(1,maxVert+1)
lastVisitedVertex = copy.deepcopy(startingVertex)
print(startingVertex)

16


# Add current vertex to solution set

In [12]:
## Add vertex to a solution attempt
solutionVertices = list()
solutionVertices.append(lastVisitedVertex)
print(solutionVertices)

[16]


In [13]:
vertexDictOfTouchedFaces[lastVisitedVertex]

[array([ 1,  7,  8, 16]),
 array([ 1,  2, 11, 16]),
 array([ 6,  7, 13, 16]),
 array([ 1, 10, 13, 16]),
 array([ 4, 11, 13, 16]),
 array([ 7, 11, 14, 16])]

# Processes Faces Touched by current vertex
# Counts faces as covered for master array
# Removes face from each vertex's untouched faces dictionary

In [14]:
## Find touched faces
currentTouchedFaces = vertexDictOfTouchedFaces[lastVisitedVertex]
#print(currentTouchedFaces)


## Count all as "covered"
    ## Add to a set of "covered faces"
##Needed to set size of array, make sure to remove later!!!!!!!!!!!!!!!!!!!!!!!!
##facesToBeAdded = array([[-1,-1,-1,-1]])
facesToBeAdded = np.ndarray(shape = (1, len(inputMatrix[0])))
facesToBeAdded.fill(-1)
facesToBeAdded = facesToBeAdded.astype(int)

for x in currentTouchedFaces:
    facesToBeAdded = np.vstack([facesToBeAdded, x])

#print(facesToBeAdded)

masterCoveredFaces = np.vstack([masterCoveredFaces, facesToBeAdded])

##Removes the size setting rows
removed = (masterCoveredFaces == -1).sum(1)
fixed = masterCoveredFaces[removed == 0, :]

   
masterCoveredFaces = copy.deepcopy(fixed)

#print(masterCoveredFaces)



## Remove from the current vertex's list of uncovered faces
for x in vertexDictOfTouchingUncoveredFaces:
    ##make temp
    ##tempXTouchingUncovered = array([[-1,-1,-1,-1]])
    ##print(tempXTouchingUncovered)
    
    tempXTouchingUncovered = np.ndarray(shape = (1, len(inputMatrix[0])))
    tempXTouchingUncovered.fill(-1)
    tempXTouchingUncovered = tempXTouchingUncovered.astype(int)
    
    
    for y in vertexDictOfTouchingUncoveredFaces[x]:
        ## make inZ
        inZ = False
        for z in vertexDictOfTouchedFaces[lastVisitedVertex]:
            if np.array_equal(y,z):
                #Set inZ to true
                inZ = True
                #print("These are equal")
        ## if not inZ, add to temp
        if not inZ:
            tempXTouchingUncovered = np.vstack([tempXTouchingUncovered, y])
    ## fix temp
        #print(tempXTouchingUncovered)
        #print("Me before fixing: ", tempXTouchingUncovered)
        removed = (tempXTouchingUncovered == -1).sum(1)
        fixed = tempXTouchingUncovered[removed == 0, :]
        #print(x, fixed)
        tempXTouchingUncovered = copy.deepcopy(fixed)
        
    
    removed = (tempXTouchingUncovered == -1).sum(1)
    fixed = tempXTouchingUncovered[removed == 0, :]
    #print(x, fixed)
    tempXTouchingUncovered = copy.deepcopy(fixed)
        
    ## make temp x's uncovered touching
    #print(x,tempXTouchingUncovered)
    vertexDictOfTouchingUncoveredFaces[x] = copy.deepcopy(tempXTouchingUncovered)
    #print("this",vertexDictOfTouchingUncoveredFaces[x])
                                       
        
        
print(vertexDictOfTouchingUncoveredFaces)

{1: array([[ 1,  2,  3, 10],
       [ 1,  2,  8,  9],
       [ 1,  8, 10, 15]]), 2: array([[ 1,  2,  3, 10],
       [ 2,  3,  4, 11],
       [ 1,  2,  8,  9],
       [ 2,  3,  9, 12],
       [ 2,  9, 11, 14]]), 3: array([[ 1,  2,  3, 10],
       [ 2,  3,  4, 11],
       [ 3,  4,  5, 12],
       [ 2,  3,  9, 12],
       [ 3,  4, 10, 13],
       [ 3, 10, 12, 15]]), 4: array([[ 2,  3,  4, 11],
       [ 3,  4,  5, 12],
       [ 4,  5,  6, 13],
       [ 3,  4, 10, 13],
       [ 4,  5, 11, 14]]), 5: array([[ 3,  4,  5, 12],
       [ 4,  5,  6, 13],
       [ 5,  6,  7, 14],
       [ 4,  5, 11, 14],
       [ 5,  6, 12, 15],
       [ 5,  9, 12, 14]]), 6: array([[ 4,  5,  6, 13],
       [ 5,  6,  7, 14],
       [ 6,  7,  8, 15],
       [ 5,  6, 12, 15],
       [ 6, 10, 13, 15]]), 7: array([[ 5,  6,  7, 14],
       [ 6,  7,  8, 15],
       [ 7,  8,  9, 14]]), 8: array([[ 6,  7,  8, 15],
       [ 1,  2,  8,  9],
       [ 7,  8,  9, 14],
       [ 1,  8, 10, 15],
       [ 8,  9, 12, 15]]), 9: array(

# Removes faces touched by current vertex from master uncovered array

In [15]:
## Remove touched faces from "uncovered" list of faces
## Removing is hard when not sure which row location removing from.
## Maybe try just rewriting without the covered ones?

#tempUncoveredFaces = array([[-1,-1,-1,-1]])
tempUncoveredFaces = np.ndarray(shape = (1, len(inputMatrix[0])))
tempUncoveredFaces.fill(-1)
tempUncoveredFaces = tempUncoveredFaces.astype(int)
print(vertexDictOfTouchingUncoveredFaces)

for x in masterUncoveredFaces:
    inY = False
    for y in vertexDictOfTouchedFaces[lastVisitedVertex]:
        #print(vertexDictOfTouchedFaces[lastVisitedVertex])
        if np.array_equal(x,y):
            inY = True
    if not inY:
        tempUncoveredFaces = np.vstack([tempUncoveredFaces,x])
#print(tempUncoveredFaces)

removed = (tempUncoveredFaces == -1).sum(1)
fixed = tempUncoveredFaces[removed == 0, :]

masterUncoveredFaces = copy.deepcopy(fixed)

print("Faces Not Yet Covered") 
print(masterUncoveredFaces)
print("")
print("Faces covered") 
print(masterCoveredFaces)


{1: array([[ 1,  2,  3, 10],
       [ 1,  2,  8,  9],
       [ 1,  8, 10, 15]]), 2: array([[ 1,  2,  3, 10],
       [ 2,  3,  4, 11],
       [ 1,  2,  8,  9],
       [ 2,  3,  9, 12],
       [ 2,  9, 11, 14]]), 3: array([[ 1,  2,  3, 10],
       [ 2,  3,  4, 11],
       [ 3,  4,  5, 12],
       [ 2,  3,  9, 12],
       [ 3,  4, 10, 13],
       [ 3, 10, 12, 15]]), 4: array([[ 2,  3,  4, 11],
       [ 3,  4,  5, 12],
       [ 4,  5,  6, 13],
       [ 3,  4, 10, 13],
       [ 4,  5, 11, 14]]), 5: array([[ 3,  4,  5, 12],
       [ 4,  5,  6, 13],
       [ 5,  6,  7, 14],
       [ 4,  5, 11, 14],
       [ 5,  6, 12, 15],
       [ 5,  9, 12, 14]]), 6: array([[ 4,  5,  6, 13],
       [ 5,  6,  7, 14],
       [ 6,  7,  8, 15],
       [ 5,  6, 12, 15],
       [ 6, 10, 13, 15]]), 7: array([[ 5,  6,  7, 14],
       [ 6,  7,  8, 15],
       [ 7,  8,  9, 14]]), 8: array([[ 6,  7,  8, 15],
       [ 1,  2,  8,  9],
       [ 7,  8,  9, 14],
       [ 1,  8, 10, 15],
       [ 8,  9, 12, 15]]), 9: array(

# Find all vertices with highest uncovered faces

In [16]:
## Finding Next Vertex
    ## Find all vertices that touch maximum number of uncovered faces
maxUntouched = 0
maxUntouchedVerts = []
for x in vertexDictOfTouchingUncoveredFaces:
    if len(vertexDictOfTouchingUncoveredFaces[x]) > maxUntouched:
        maxUntouched = len(vertexDictOfTouchingUncoveredFaces[x])
    #print(x, len(vertexDictOfTouchingUncoveredFaces[x]))
    #print("max ", maxUntouched )
    
for x in vertexDictOfTouchingUncoveredFaces:
    if len(vertexDictOfTouchingUncoveredFaces[x]) == maxUntouched:
            maxUntouchedVerts.append(x)
print(maxUntouchedVerts)
print(maxUntouchedVerts[0])
    
    
    ## Calculate which of vertices in previous step are closest to a covered face 
if len(maxUntouchedVerts) == 1:
    lastVisitedVertex = maxUntouchedVerts[0]
elif len(maxUntouchedVerts) == 0:
    print("SOMETHING IS VERY WRONG HERE, NO MAXIMIZED VERTS")
else:
    print("we are calculating")
    lastVisitedVertex = findNextVertex(maxUntouchedVerts)
    
    
print("The next vertex is" , lastVisitedVertex)

[3, 5, 9, 12, 15]
3
we are calculating
The next vertex is 3


# Check if all faces covered, if yes, print out solution, covered and uncovered

In [17]:
## Check if all faces covered
print(np.array_equal(inputMatrix, masterCoveredFaces))

## If yes, output list of vertices as solution
## Covered face must be 1 version of all, and uncovered must be empty
if np.array_equal(inputMatrix, masterCoveredFaces) and len(masterUncoveredFaces) == 0:
    print("Solution Set")
    print(solutionVertices)
    print(len(solutionVertices), " vertices in solution set")
    print( " ")
    print("Uncoverd faces")
    print(masterUncoveredFaces)
    print( " ")
    print("Covered Faces")
    print(masterCoveredFaces)

else:
    lastVisitedVertex = findNextVertex(maxUntouchedVerts)


    
print(lastVisitedVertex)
    ## If no, select next Vertex

False
3
